In [ ]:
!pip install cloud-tpu-client==0.10 torch==1.9.0 torchvision==0.10.0 torchtext==0.10.0 torchaudio==0.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!pip install catalyst==21.09
!pip install datasets==1.11.0 transformers==4.10.0

# BERT Embedding

Привет! В этом домашнем задании вы обучите модель BERT для получения эмбеддингов предложений. Обучать BERT может потребовать много ресурсов, поэтому здесь предлагается использовать TPU в качестве ускорителя вычислений.

!!!
**Внимание**

В этой работе запрещается использовать модель, которая была предобученная на **задачу эмбеддингов**. (Next Sentence Prediction разрешена)

В этой работе запрещается использовать напрямую библиотеку **sentence-transformer**. 

!!!

Статьи, которые помогут вам придумать улучшения для модели/процесса обучения/etc:
- https://arxiv.org/pdf/1908.10084.pdf
- https://ai.googleblog.com/2020/08/language-agnostic-bert-sentence.html



In [ ]:
from torch import __version__ as torch_version
from catalyst import __version__ as catalyst_version
from catalyst import SETTINGS
print(torch_version, catalyst_version, SETTINGS.xla_required)

In [ ]:
import transformers
import datasets
from datetime import datetime
import hashlib
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from sklearn.cluster import KMeans

from transformers import AutoModelForSequenceClassification, AutoTokenizer, get_scheduler

from catalyst import dl
from catalyst.contrib.nn import ResidualBlock
from catalyst.data import transforms

Тут описание обучения модели и получения эмбеддингов из модели. Тут не использована сила библиотеки transformers, этот код можно ускорить и упростить: https://github.com/huggingface/transformers/blob/c9d2cf855a38addaf036fa010e92cc8563a0ce07/src/transformers/models/bert/modeling_bert.py#L627


Пояснение по коду:
- forward используется для обучения модели
- get_embedding используется для инференса модели

В этом примере используется модель [RuBERT-tiny](https://huggingface.co/cointegrated/rubert-tiny). Можете попробовать любую другую предобученную модель.

В некоторых моделях есть заранее предобученный pooler. Иногда полезно исползовать его. Можете менять код в ячейке ниже на свое усмотрение.

In [ ]:
class SimBERT(nn.Module):
    def __init__(self, model_name: str):
        super().__init__()

        self.model = transformers.BertModel.from_pretrained(model_name)

    def forward(self, inputs):
        x_embs = self.model(input_ids=inputs["x_input_ids"])
        y_embs = self.model(input_ids=inputs["y_input_ids"])

        x_emb = self.pooling(x_embs)
        y_emb = self.pooling(y_embs)

        return nn.functional.cosine_similarity(x_emb, y_emb)

    def get_embedding(self, x):
        return self.pooling(self.model(**x))

    def pooling(self, x):
        return x["last_hidden_state"][:, 0]

Тут находится Catalyst Runner для обучения модели. Он помогает работать с TPU также, как с GPU.

Самые важные детали CustomRunner:

- get_loaders -- Получение и обработки данных
- get_optimizer/get_lr_sheduler -- Получение оптимизатора и lr_sheduler-а

Здесь показано обучения не датасете [XNLI-ru](https://huggingface.co/datasets/xnli). Можете попробовать любой другой набор данных.

In [ ]:
class CustomRunner(dl.IRunner):
    def __init__(self, logdir):
        super().__init__()
        self._logdir = logdir

    def get_engine(self):
        return dl.DistributedXLAEngine()

    def get_loggers(self):
        return {
            "console": dl.ConsoleLogger(),
            "csv": dl.CSVLogger(logdir=self._logdir),
            "tensorboard": dl.TensorboardLogger(logdir=self._logdir),
        }

    @property
    def stages(self):
        return ["train"]

    def get_stage_len(self, stage: str) -> int:
        return 1

    def get_loaders(self, stage: str):
        xnli =  datasets.load_dataset("xnli", "ru")
        tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")

        def process(examples):
            x = tokenizer(
                examples["premise"],
                max_length=128,
                truncation=True,
                padding="max_length",
            )
            y = tokenizer(
                examples["hypothesis"],
                max_length=128,
                truncation=True,
                padding="max_length",
            )
            result = {f"x_{k}": v for k, v in x.items()}
            result.update({f"y_{k}": v for k, v in y.items()})
            return result

        encoded_datasets = xnli.map(process, batched=True)
        encoded_datasets = encoded_datasets.map(lambda x: {"labels": 1.0 - x["label"]})
        encoded_datasets.set_format(
            type="torch",
            columns=[
                "x_input_ids",
                "x_attention_mask",
                "y_input_ids",
                "y_attention_mask",
                "labels",
            ]
        )
        
        train_data = encoded_datasets["train"]
        valid_data = encoded_datasets["validation"]

        if self.engine.is_ddp:
            train_sampler = torch.utils.data.distributed.DistributedSampler(
                train_data,
                num_replicas=self.engine.world_size,
                rank=self.engine.rank,
                shuffle=True
            )
            valid_sampler = torch.utils.data.distributed.DistributedSampler(
                valid_data,
                num_replicas=self.engine.world_size,
                rank=self.engine.rank,
                shuffle=False
            )
        else:
            train_sampler = valid_sampler = None

        self.train_loader_len = len(DataLoader(train_data, batch_size=64, sampler=train_sampler))

        return {
            "train": DataLoader(train_data, batch_size=256, sampler=train_sampler),
            "valid": DataLoader(valid_data, batch_size=64, sampler=valid_sampler),
        }

    def get_model(self, stage: str):
        model = self.model \
            if self.model is not None \
            else SimBERT("cointegrated/rubert-tiny")
        return model

    def get_criterion(self, stage: str):
        return nn.MSELoss()

    def get_optimizer(self, stage: str, model):
        return torch.optim.Adam(model.parameters(), lr=1e-4)

    def get_scheduler(self, stage: str, optimizer):
        scheduler = get_scheduler(
            "linear",
            optimizer=optimizer,
            num_warmup_steps=int(0.05 * self.train_loader_len) * self.stage_epoch_len,
            num_training_steps=self.train_loader_len * self.stage_epoch_len
        )
        return scheduler

    def get_callbacks(self, stage: str):
        return {
            "criterion": dl.CriterionCallback(
                input_key="logits", target_key="labels", metric_key="loss"
            ),
            "optimizer": dl.OptimizerCallback(metric_key="loss"),
            "scheduler": dl.SchedulerCallback(loader_key="valid", metric_key="loss", mode="batch"),
            "checkpoint": dl.CheckpointCallback(
                self._logdir,
                loader_key="valid",
                metric_key="loss",
                minimize=False,
                save_n_best=1,
            ),
            "tqdm": dl.TqdmCallback(),
        }

    def handle_batch(self, batch):
        outputs = self.model(batch)

        self.batch = {
            "labels": batch["labels"],
            "logits": outputs,
        }

Запускаем логгер

In [ ]:
logdir = f"logs/{datetime.now().strftime('%Y%m%d-%H%M%S')}"
%load_ext tensorboard
%tensorboard --logdir=./logs/

Запускаем обучение эмбеддингов.

In [ ]:
runner = CustomRunner(logdir)
runner.run()

Применняем обученные эмбеддинги на тестовой выборке.

In [ ]:
test_table = pd.read_csv("test.csv")
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")

model = runner.get_model(stage=1)
embs = []
with torch.no_grad():
    model.eval()
    for _, row in tqdm(test_table.iterrows(), total=test_table.shape[0]):
        x = tokenizer.batch_encode_plus(
            [row["text"]],
            max_length=128,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )
        emb = model.get_embedding(x)
        embs.append(
            {
                "id": row["id"],
                "emb": emb.detach().cpu()[0].numpy()
            }
        )

Для получения сабмита требуется дописать код ;-)
Что осталось сделать:
- Найти эмбеддинги для заголовков из fewshot.csv
- Разметить остальные заголовки из test.csv

In [ ]:
# ваш код